In [1]:
import torch
print(torch.__version__)
from datasets import load_dataset, Sequence, Features, Value

2.8.0+cu126


In [2]:
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
print(dataset.num_rows)
print(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


36718
Dataset({
    features: ['text'],
    num_rows: 36718
})


In [3]:
print(dataset[0])
print(dataset[1])
print(dataset[-2])
print(dataset[-1])

{'text': ''}
{'text': ' = Valkyria Chronicles III = \n'}
{'text': ' Common starlings are trapped for food in some Mediterranean countries . The meat is tough and of low quality , so it is casseroled or made into pâté . One recipe said it should be stewed " until tender , however long that may be " . Even when correctly prepared , it may still be seen as an acquired taste . \n'}
{'text': ''}


In [4]:
def tokenize_row(row):
    # basic whitespace tokenizer for demo
    tokens = row["text"].lower().split()
    return {"tokens": tokens}

In [5]:
dataset[1]["text"]

' = Valkyria Chronicles III = \n'

In [6]:
dataset = dataset.map(tokenize_row)

In [7]:
vocab = list(set([tok for row in dataset["tokens"] for tok in row]))
print(f"Vocab size: {len(vocab)}")
print(vocab)


Vocab size: 66649
['outdated', 'naturalist', 'yeah', 'nlds', 'quercus', 'sumo', 'oxirane', 'icefrog', 'muslim', 'polchester', 'dunlop', 'encapture', 'wayland', 'harekâtı', 'wandering', 'dregs', 'intestinal', 'budget', 'asking', 'siad', 'fleeting', 'croft', 'evolution', 'nonconformity', 'finals', 'lourenço', 'hovered', 'compelled', 'resultant', 'euchromatic', 'thew', 'chronicling', 'diffident', 'saints', 'gilford', 'pomz', 'gérard', 'chancel', 'solyaris', 'diarmata', 'planets', 'pushy', 'commanders', 'cocaine', 'cornfields', 'arwel', '512', 'commensurability', 'pitres', 'dowd', 'd.c', 'bind', 'cannons', 'sweet', 'crumpled', 'brackenburn', 'instantly', 'cyclones', 'portrush', 'gracen', 'meyerbeer', 'stall', 'repeals', 'injecting', 'takeshi', 'coincidental', 'rippling', 'zetra', 'nagwar', 'luton', 'qg3', 'knut', 'republic', 'eóganachta', 'paddan', 'parvarit', 'connotation', 'redeemer', 'shona', 'tv5', 'fits', 'hemodialysis', 'prost', 'полосатый', 'arnaud', 'socializes', 'ahmedabad', 'glas

In [8]:
if "valkyria" in vocab:
    print("Valkyria is in the vocab")

Valkyria is in the vocab


In [9]:
id2tok = dict(enumerate(vocab))
list(id2tok.items())[:5]

[(0, 'outdated'), (1, 'naturalist'), (2, 'yeah'), (3, 'nlds'), (4, 'quercus')]

In [10]:
tok2id = {tok: i for (i,tok) in id2tok.items()}
list(tok2id.items())[:5]

[('outdated', 0), ('naturalist', 1), ('yeah', 2), ('nlds', 3), ('quercus', 4)]

In [11]:
WINDOW_WIDTH = 10
def windowizer(row, wsize=WINDOW_WIDTH):
  doc = row['tokens']
  out = []
  context_ids = []
  for i, wd in enumerate(doc):
    target = tok2id[wd]
    window = [
        i + j for j in range(-wsize, wsize+1, 1)
        if (i + j >=0) and (i + j < len(doc)) and (j != 0)
    ]

    # context_ids = [
    #         int(tok2id[doc[w]]) for w in window
    #         if doc[w] in tok2id
    #     ]
    #context_ids = []
    for j in window:
            cid = tok2id.get(doc[j])
            if cid is not None:
                out.append([target, int(cid)])
                #context_ids.append(int(cid))

    #out.append([target, context_ids])
  row['window'] = out

  return row




In [12]:
dataset["text"]

Column(['', ' = Valkyria Chronicles III = \n', '', ' Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . \n', " The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game mor

In [13]:
dataset["tokens"]

Column([[], ['=', 'valkyria', 'chronicles', 'iii', '='], [], ['senjō', 'no', 'valkyria', '3', ':', 'unrecorded', 'chronicles', '(', 'japanese', ':', '戦場のヴァルキュリア3', ',', 'lit', '.', 'valkyria', 'of', 'the', 'battlefield', '3', ')', ',', 'commonly', 'referred', 'to', 'as', 'valkyria', 'chronicles', 'iii', 'outside', 'japan', ',', 'is', 'a', 'tactical', 'role', '@-@', 'playing', 'video', 'game', 'developed', 'by', 'sega', 'and', 'media.vision', 'for', 'the', 'playstation', 'portable', '.', 'released', 'in', 'january', '2011', 'in', 'japan', ',', 'it', 'is', 'the', 'third', 'game', 'in', 'the', 'valkyria', 'series', '.', 'employing', 'the', 'same', 'fusion', 'of', 'tactical', 'and', 'real', '@-@', 'time', 'gameplay', 'as', 'its', 'predecessors', ',', 'the', 'story', 'runs', 'parallel', 'to', 'the', 'first', 'game', 'and', 'follows', 'the', '"', 'nameless', '"', ',', 'a', 'penal', 'military', 'unit', 'serving', 'the', 'nation', 'of', 'gallia', 'during', 'the', 'second', 'europan', 'war', 'w

In [14]:

features = Features({
    "text" : Value("string"),
    "tokens": Sequence(Value("string")),
    "window": Sequence(Sequence(Value("int64")))  # [[int, int], ...]
})
dataset = dataset.map(windowizer, features=features, load_from_cache_file=False)

#dataset = dataset.map(windowizer, features = features)

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

In [15]:
dataset

Dataset({
    features: ['text', 'tokens', 'window'],
    num_rows: 36718
})

In [16]:
dataset[5]

{'text': " It met with positive sales in Japan , and was praised by both Japanese and western critics . After release , it received downloadable content , along with an expanded edition in November of that year . It was also adapted into manga and an original video animation series . Due to low sales of Valkyria Chronicles II , Valkyria Chronicles III was not localized , but a fan translation compatible with the game 's expanded edition was released in 2014 . Media.Vision would return to the franchise with the development of Valkyria : Azure Revolution for the PlayStation 4 . \n",
 'tokens': ['it',
  'met',
  'with',
  'positive',
  'sales',
  'in',
  'japan',
  ',',
  'and',
  'was',
  'praised',
  'by',
  'both',
  'japanese',
  'and',
  'western',
  'critics',
  '.',
  'after',
  'release',
  ',',
  'it',
  'received',
  'downloadable',
  'content',
  ',',
  'along',
  'with',
  'an',
  'expanded',
  'edition',
  'in',
  'november',
  'of',
  'that',
  'year',
  '.',
  'it',
  'was'

In [24]:
# tok2id = {'the': 0, 'cat': 1, 'sat': 2, 'on': 3, 'mat': 4}
# WINDOW_WIDTH = 2
# tokens = ['the', 'cat', 'sat', 'on', 'mat']

def skip_grams(tokens, window_width=WINDOW_WIDTH):

  pairs = []
  for i, wd in enumerate(tokens):
    target = tok2id.get(wd)
    print(target)
    window = [
        i+j for j in range(-window_width, window_width+1, 1)
        if (i+j >= 0) and (i+j < len(tokens)) and (j != 0)
    ]
    pairs.extend([(target, tok2id[tokens[w]]) for w in window])

  return pairs
# pairs = skip_grams(tokens)
# print(pairs)

0
1
2
3
4
[(0, 1), (0, 2), (1, 0), (1, 2), (1, 3), (2, 0), (2, 1), (2, 3), (2, 4), (3, 1), (3, 2), (3, 4), (4, 2), (4, 3)]


In [23]:
pairs = dataset.map(skip_grams)


print(pairs)

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

KeyError: 1